## Проверка данных Цифромеда

In [1]:
import pandas as pd
import numpy as np
import os
import random

# игнорирование предупреждения о формате при импорте из excel
import warnings

warnings.simplefilter("ignore")
# пути и списки файлов
# файлы берём непосредственно из хранилища нового бота посещений
e_path = "C:/work/new_bot/new/e"
s_path = "C:/work/new_bot/new/s"
dm_path = "C:/work/new_bot/new/dm"
e_file_list = os.listdir(e_path)
s_file_list = os.listdir(s_path)
dm_file_list = os.listdir(dm_path + "/" + max(os.listdir(dm_path)))

In [57]:
# типы ошибок
t_error_list = [
    "Ошибка валидации данных пациента",
    "Ошибка «РМИС не ответил»",
    "Внутренняя ошибка РМИС при создании записи",
    "Сервис МИС приостановлен",
    "Ошибка валидации схемы",
    "Время уже занято другим пациентом",
    # "Пациент уже записан к специалисту в этот день",
    "Услуга посмертно не оказывается",
    # "Пациент уже записан на это время к другому специалисту",
    # "Выбранное время доступно только для записи пациентов определенного возраста",
    "Время начала приема уже прошло",
]
tdb_error_list = [
    "Ошибка валидации данных пациента",
    "Ошибка «РМИС не ответил»",
    "Внутренняя ошибка РМИС при создании записи",
    "Сервис МИС приостановлен",
    "Ошибка валидации схемы",
    # "Время уже занято другим пациентом",
    # "Услуга посмертно не оказывается",
    # "Время начала приема уже прошло",
]
b_list = [
    "Пациент уже записан на это время к другому специалисту",
    "Пациент уже записан к специалисту в этот день",
    "Выбранное время доступно только для записи пациентов определенного возраста",
]
o_error_list = [
    "Значение не найдено",
]
r_error_list = [
    "Ошибка «СМЭВ-timeout»",
    "Внутренняя ошибка концентратора при создании записи",
    "Ошибка во входных параметрах при записи на прием",
]
any_error_list = [
    "Ошибка доставки формы при записи на прием",
    "Истекло время сессии пользователя",
    "Ошибка соответствия ответа запросу МИС",
]
draft_list = ["Ошибка отсутсвует"]

In [2]:
# for file_name in e_file_list:
#     print(file_name)
#     df_tmp = pd.read_csv(e_path + "/" + file_name, sep=";")
#     break

01.11.csv


In [71]:
for file_name in e_file_list:
    df_tmp = pd.read_csv(e_path + "/" + file_name, sep=";")
    print(f"\nФайл {file_name}:")
    print(f"\t\tвсего строк: {df_tmp.shape[0]}")
    cnt_c = "верно" if df_tmp.shape[1] == 13 else "ошибка"
    print(f"\t\tвсего столбцов: {df_tmp.shape[1]} ({cnt_c})")
    print("\tПропуски в столбцах, где их быть не должно:")
    df_nonan = df_tmp[
        [
            "session_id",
            "Название субъекта РФ",
            "step_id",
            "step_name",
            "error_code",
            "final_state",
            "create_ts",
            "Тип записи",
            "referral_flg",
        ]
    ]
    n = 0
    for column in df_nonan.columns:
        if df_nonan[column].isnull().sum() != 0:
            print(
                f"\t\tстолбец '{column}':\t\t {df_nonan[column].isnull().sum()} ({round(df_nonan[column].isnull().sum()*100/df_nonan.shape[0], 2)} %)"
            )
            n += 1
    if n == 0:
        print("\t\tотсутствуют")
    print(
        "\tОшибки данных (количество и доля от числа строк на соответствующем этапе):"
    )
    # число строк на соответствующих этапах
    len_ValidatePerson = df_tmp[df_tmp["step_name"] == "ValidatePerson"].shape[0]
    len_MO = df_tmp[df_tmp["step_name"] == "MO"].shape[0]
    len_ServiceOrSpecs = df_tmp[df_tmp["step_name"] == "ServiceOrSpecs"].shape[0]
    len_Slot = df_tmp[df_tmp["step_name"] == "Slot"].shape[0]
    len_Book = df_tmp[df_tmp["step_name"] == "Book"].shape[0]

    """
    На этапе валидации пациента
    есть данные oid МО
    ИЛИ есть данные о названии МО
    ИЛИ есть данные о специальности врача
    ИЛИ есть данные о слоте
    """
    df_ValidatePerson = df_tmp[
        (df_tmp["step_name"] == "ValidatePerson")
        & (
            df_tmp["mo_oid"].notnull()
            | df_tmp["mo_name"].notnull()
            | df_tmp["Post_name"].notnull()
            | df_tmp["slot_ts"].notnull()
        )
    ]
    """
    На этапе выбора МО
    есть данные oid МО
    ИЛИ есть данные о названии МО
    ИЛИ есть данные о специальности врача
    ИЛИ есть данные о слоте
    """
    df_MO = df_tmp[
        (df_tmp["step_name"] == "MO")
        & (
            df_tmp["mo_oid"].notnull()
            | df_tmp["mo_name"].notnull()
            | df_tmp["Post_name"].notnull()
            | df_tmp["slot_ts"].notnull()
        )
    ]
    """
    На этапе выбора специальности врача
    нет данных об oid МО
    ИЛИ нет данных о названии МО
    ИЛИ есть данные о специальности врача
    ИЛИ есть данные о слоте
    """
    df_ServiceOrSpecs = df_tmp[
        (df_tmp["step_name"] == "ServiceOrSpecs")
        & (
            ~df_tmp["mo_oid"].notnull()
            | ~df_tmp["mo_name"].notnull()
            | df_tmp["Post_name"].notnull()
            | df_tmp["slot_ts"].notnull()
        )
    ]
    """"
    На этапе выбора слота
    нет данных об oid МО
    ИЛИ нет данных о названии МО
    ИЛИ нет данных о специальтности врача
    ИЛИ есть данные о времени слота
    """
    df_Slot = df_tmp[
        (df_tmp["step_name"] == "Slot")
        & (
            ~df_tmp["mo_oid"].notnull()
            | ~df_tmp["mo_name"].notnull()
            | ~df_tmp["Post_name"].notnull()
            | df_tmp["slot_ts"].notnull()
        )
    ]
    """
    На этапе записи
    нет любых данных (ИЛИ)
    """
    df_Book = df_tmp[
        (df_tmp["step_name"] == "Book")
        & (
            ~df_tmp["mo_oid"].notnull()
            | ~df_tmp["mo_name"].notnull()
            | ~df_tmp["Post_name"].notnull()
            | ~df_tmp["slot_ts"].notnull()
        )
    ]

    if df_ValidatePerson.shape[0] >= 0:
        print(
            f"\t\tValidatePerson_uncorrected:\t{df_ValidatePerson.shape[0]}\t({round(df_ValidatePerson.shape[0]*100/len_ValidatePerson, 2)} %)"
        )
    if df_MO.shape[0] >= 0:
        print(
            f"\t\tMO_uncorrected:\t\t\t{df_MO.shape[0]}\t({round(df_MO.shape[0]*100/len_MO, 2)} %)"
        )
    if df_ServiceOrSpecs.shape[0] >= 0:
        print(
            f"\t\tServiceOrSpecs_uncorrected:\t{df_ServiceOrSpecs.shape[0]}\t({round(df_ServiceOrSpecs.shape[0]*100/len_ServiceOrSpecs, 2)} %)"
        )
    if df_Slot.shape[0] >= 0:
        print(
            f"\t\tSlot_uncorrected:\t\t{df_Slot.shape[0]}\t({round(df_Slot.shape[0]*100/len_Slot, 2)} %)"
        )
    if df_Book.shape[0] >= 0:
        print(
            f"\t\tBook_uncorrected:\t\t{df_Book.shape[0]}\t({round(df_Book.shape[0]*100/len_Book, 2)} %)"
        )
    if (
        df_ValidatePerson.shape[0]
        + df_MO.shape[0]
        + df_ServiceOrSpecs.shape[0]
        + df_Slot.shape[0]
        + df_Book.shape[0]
        == 0
    ):
        print("\t\tотсутствуют")
    df_errors = pd.DataFrame(
        df_tmp[df_tmp["Тип записи"] == 3].groupby("final_state")["session_id"].count()
    ).reset_index()
    print("\tКоличество ошибок записи на приём (тип записи 'К врачу'):")
    print(
        f'\t\tорганизационные ошибки:\t\t{df_errors[df_errors["final_state"].isin(o_error_list)]["session_id"].sum()}'
    )
    print(
        f'\t\tтехнические ошибки:\t\t{df_errors[df_errors["final_state"].isin(t_error_list)]["session_id"].sum()} ({df_errors[df_errors["final_state"].isin(tdb_error_list)]["session_id"].sum()})'
    )
    print(
        f'\t\tошибки ФЭР:\t\t\t{df_errors[df_errors["final_state"].isin(r_error_list)]["session_id"].sum()}'
    )
    print(
        f'\t\tпрочие ошибки:\t\t\t{df_errors[df_errors["final_state"].isin(any_error_list)]["session_id"].sum()}'
    )
    print(
        f'\t\tчерновики:\t\t\t{df_errors[df_errors["final_state"].isin(draft_list)]["session_id"].sum()}'
    )
    print(
        f'\t\tбизнес-правила:\t\t\t{df_errors[df_errors["final_state"].isin(b_list)]["session_id"].sum()}'
    )
    print("\tСубъекты, не отображаемые на дашборде:")
    df_nosub = df_tmp[
        (df_tmp["Название субъекта РФ"] == "Федеральное медико-биологическое агентство")
        | (df_tmp["Название субъекта РФ"] == "г. Байконур")
    ][["Название субъекта РФ", "final_state"]]
    if df_nosub.empty:
        print("\t\tотсутствуют")
    else:
        print(df_nosub.to_string(index=False))
    # break


Файл 01.11.csv:
		всего строк: 524928
		всего столбцов: 13 (верно)
	Пропуски в столбцах, где их быть не должно:
		отсутствуют
	Ошибки данных (количество и доля от числа строк на соответствующем этапе):
		ValidatePerson_uncorrected:	4	(0.01 %)
		MO_uncorrected:			0	(0.0 %)
		ServiceOrSpecs_uncorrected:	3620	(3.72 %)
		Slot_uncorrected:		2646	(2.38 %)
		Book_uncorrected:		40	(0.22 %)
	Количество ошибок записи на приём (тип записи 'К врачу'):
		организационные ошибки:		68658
		технические ошибки:		53821 (53821)
		ошибки ФЭР:			27113
		прочие ошибки:			0
		черновики:			374375
		бизнес-правила:			0
	Субъекты, не отображаемые на дашборде:
		отсутствуют

Файл 02.11.csv:
		всего строк: 496113
		всего столбцов: 13 (верно)
	Пропуски в столбцах, где их быть не должно:
		столбец 'Название субъекта РФ':		 1 (0.0 %)
	Ошибки данных (количество и доля от числа строк на соответствующем этапе):
		ValidatePerson_uncorrected:	1	(0.0 %)
		MO_uncorrected:			0	(0.0 %)
		ServiceOrSpecs_uncorrected:	3564	(3.6